## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Elizabeth,ZA,-33.9180,25.5701,59.00,94,90,4.61,fog
1,1,Busselton,AU,-33.6500,115.3333,68.00,73,21,1.99,few clouds
2,2,Murmashi,RU,68.8154,32.8115,28.40,54,0,6.71,clear sky
3,3,Teya,RU,60.3778,92.6267,28.45,78,94,7.47,overcast clouds
4,4,Beitbridge,ZW,-22.2167,30.0000,66.40,68,0,0.67,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Busselton,AU,-33.6500,115.3333,68.00,73,21,1.99,few clouds
4,4,Beitbridge,ZW,-22.2167,30.0000,66.40,68,0,0.67,clear sky
5,5,Ouahigouya,BF,13.5828,-2.4216,84.13,30,50,9.28,scattered clouds
6,6,Atuona,PF,-9.8000,-139.0333,81.00,73,14,19.66,few clouds
13,13,Cidreira,BR,-30.1811,-50.2056,71.02,71,96,19.51,overcast clouds
15,15,Tucupita,VE,9.0622,-62.0510,73.35,91,80,3.04,broken clouds
16,16,Binabalian,PH,16.4202,119.9279,83.68,68,18,2.04,few clouds
19,19,Odweyne,SO,9.4092,45.0640,68.94,64,30,9.66,scattered clouds
20,20,Alofi,NU,-19.0595,-169.9187,82.40,61,40,5.75,scattered clouds
21,21,Tura,IN,25.5198,90.2201,87.28,39,92,6.60,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                315
City                   315
Country                312
Lat                    315
Lng                    315
Max Temp               315
Humidity               315
Cloudiness             315
Wind Speed             315
Current Description    315
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0,inplace=False)
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Busselton,AU,-33.6500,115.3333,68.00,73,21,1.99,few clouds
4,4,Beitbridge,ZW,-22.2167,30.0000,66.40,68,0,0.67,clear sky
5,5,Ouahigouya,BF,13.5828,-2.4216,84.13,30,50,9.28,scattered clouds
6,6,Atuona,PF,-9.8000,-139.0333,81.00,73,14,19.66,few clouds
13,13,Cidreira,BR,-30.1811,-50.2056,71.02,71,96,19.51,overcast clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Busselton,AU,68.00,few clouds,-33.6500,115.3333,
4,Beitbridge,ZW,66.40,clear sky,-22.2167,30.0000,
5,Ouahigouya,BF,84.13,scattered clouds,13.5828,-2.4216,
6,Atuona,PF,81.00,few clouds,-9.8000,-139.0333,
13,Cidreira,BR,71.02,overcast clouds,-30.1811,-50.2056,
15,Tucupita,VE,73.35,broken clouds,9.0622,-62.0510,
16,Binabalian,PH,83.68,few clouds,16.4202,119.9279,
19,Odweyne,SO,68.94,scattered clouds,9.4092,45.0640,
20,Alofi,NU,82.40,scattered clouds,-19.0595,-169.9187,
21,Tura,IN,87.28,overcast clouds,25.5198,90.2201,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0,inplace=False)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Busselton,AU,68.00,few clouds,-33.6500,115.3333,Observatory Guest House
4,Beitbridge,ZW,66.40,clear sky,-22.2167,30.0000,PETERS MOTEL & CONFERENCE CENTRE
5,Ouahigouya,BF,84.13,scattered clouds,13.5828,-2.4216,Hotel Liberte
6,Atuona,PF,81.00,few clouds,-9.8000,-139.0333,Villa Enata
13,Cidreira,BR,71.02,overcast clouds,-30.1811,-50.2056,Hotel Castelo


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))